## POS tagging using modified Viterbi

### Data Preparation

In [38]:
#Importing libraries
import random
import time
import numpy as np
import pandas as pd
import nltk
from sklearn.model_selection import train_test_split

In [2]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

In [4]:
# first 5 tagged sentences
print(nltk_data[:5])

[[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')], [('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')], [('Rudolph', 'NOUN'), ('Agnew', 'NOUN'), (',', '.'), ('55', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), ('and', 'CONJ'), ('former', 'ADJ'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Consolidated', 'NOUN'), ('Gold', 'NOUN'), ('Fields', 'NOUN'), ('PLC', 'NOUN'), (',', '.'), ('was', 'VERB'), ('named', 'VERB'), ('*-1', 'X'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('of', 'ADP'), ('this', 'DET'), ('British', 'ADJ'), ('industrial', 'ADJ'), ('

In [96]:
# lets split the data into training and validation set in ratio 90:10
train_set, valid_set = train_test_split(nltk_data,train_size=0.95,test_size=0.05,random_state = 42)

In [97]:
# create list of train tagged words with both words and POS tags 
# creating the list of words for the valdiation set 
train_tagged_words = [tup for sent in train_set for tup in sent]
valid_words = [tup[0] for sent in valid_set for tup in sent]
print(len(train_tagged_words))
print(len(valid_words))

95589
5087


In [98]:
# some of the tagged words in train set
print("train tagged words")
print(train_tagged_words[0:10])

train tagged words
[('Bank', 'NOUN'), ('of', 'ADP'), ('New', 'NOUN'), ('England', 'NOUN'), ("'s", 'PRT'), ('shares', 'NOUN'), ('are', 'VERB'), ('traded', 'VERB'), ('*-1', 'X'), ('on', 'ADP')]


In [99]:
# getting the unique tags in the train set
unique_train_tags = {tag for word,tag in train_tagged_words}

In [100]:
print("number of unique tags in train set is %s" %(len(unique_train_tags)))
print("unique tags are %s" %(unique_train_tags))

number of unique tags in train set is 12
unique tags are {'NOUN', 'CONJ', 'X', 'ADJ', 'DET', 'ADP', 'ADV', 'PRT', 'NUM', '.', 'PRON', 'VERB'}


### Build the vanilla Viterbi based POS tagger

In [101]:
# function to generate the emission probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)
    return (count_w_given_tag, count_tag)

In [102]:
# function to generate the transmission probability
def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [103]:
# testing the function written to get the transmission probability
print(t2_given_t1('NOUN','DET'))

(5302, 8284)


In [104]:
# creating t x t transition matrix of tags
# each column is t2, each row is t1
# thus M(i, j) represents P(tj given ti)

tags_matrix = np.zeros((len(unique_train_tags), len(unique_train_tags)), dtype='float32')
for i, t1 in enumerate(list(unique_train_tags)):
    for j, t2 in enumerate(list(unique_train_tags)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

In [105]:
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(unique_train_tags), index=list(unique_train_tags))

In [106]:
tags_df

,NOUN,CONJ,X,ADJ,DET,ADP,ADV,PRT,NUM,.,PRON,VERB
NOUN,0.264632,0.041936,0.029136,0.012289,0.013310,0.176275,0.016884,0.043832,0.009627,0.239179,0.004923,0.147978
CONJ,0.349132,0.000469,0.007977,0.116847,0.121539,0.054435,0.055842,0.004693,0.042234,0.034256,0.058658,0.153918
X,0.061345,0.010357,0.076482,0.016571,0.055131,0.142925,0.025175,0.185787,0.002709,0.163799,0.056087,0.203633
ADJ,0.696725,0.016949,0.021392,0.066645,0.005101,0.078986,0.004608,0.010861,0.020405,0.065328,0.000658,0.012342
DET,0.640029,0.000483,0.045509,0.204973,0.005311,0.009054,0.012313,0.000241,0.021970,0.017986,0.003742,0.038387
ADP,0.321776,0.000856,0.034029,0.105297,0.326378,0.017228,0.013162,0.001498,0.062921,0.039486,0.069128,0.008240
ADV,0.030897,0.006312,0.023588,0.130233,0.067110,0.119601,0.081063,0.013621,0.030565,0.136877,0.015615,0.344518
PRT,0.242563,0.002288,0.013403,0.086303,0.100360,0.021576,0.010134,0.001635,0.058516,0.041517,0.018960,0.402746
NUM,0.355338,0.013381,0.206661,0.034196,0.003271,0.034790,0.002974,0.027951,0.184062,0.118347,0.001487,0.017544
.,0.223152,0.057924,0.026623,0.044972,0.173502,0.091114,0.052078,0.002339,0.080500,0.093812,0.065389,0.088505


In [107]:
# code for vanilla viterbi algo
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [108]:
# testing the vanilla viterbi algo on the validation set
# Let's test our Viterbi algorithm on a few sample sentences of test dataset
random.seed(1234)

# choose random 5 sents
rand_int = [random.randint(1,len(valid_set)) for x in range(5)]

# list of sents
test_run = [valid_set[i] for i in rand_int]

# list of tagged words
test_run_base = [tup for sent in test_run for tup in sent]

# list of untagged words
test_tagged_words = [tup[0] for sent in test_run for tup in sent]

In [109]:
# tagging the validation sentences
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start

print("Time taken(seconds): %s" %(difference))

# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 

accuracy = len(check)/len(tagged_seq)
print('Vanilla Viterbi Algorithm Accuracy: %s' %(accuracy*100))

Time taken(seconds): 14.586943864822388
Vanilla Viterbi Algorithm Accuracy: 92.78350515463917


In [110]:
# incorrectly tagged words 
[j for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0] != j[1]]

[(('develops', 'NOUN'), ('develops', 'VERB')),
 (('markets', 'NOUN'), ('markets', 'VERB')),
 (('low-cost', 'NOUN'), ('low-cost', 'ADJ')),
 (('pre-1917', 'NOUN'), ('pre-1917', 'ADJ')),
 (('test', 'VERB'), ('test', 'NOUN')),
 (('out', 'PRT'), ('out', 'ADV')),
 (('copied', 'NOUN'), ('copied', 'VERB'))]

### Solve the problem of unknown words

We can see most of the unknown words are tagged as Noun. and Noun is the first tag in the taglist and is assigned if unknown words
is identified. since emission probability of the unkown word is 0

First solution to unknown words 
* <b>consider the transmission probability in case the unknown words is encounterd</b> as for the unknown words the emission probability 
is 0 so the multiplication of emission probability and transmission probability is 0

In [111]:
# mofdifing the vanilla viterbi algo to consider the transmission probability if the emission probablity is 0
def Viterbi_Mod1(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        transmission_prob =[] # storing transisition probabilities
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            transmission_prob.append(transition_p)
        pmax = max(p)
        state_max = T[p.index(pmax)] 
      
        # if probability is zero ie. there is unknown word then consider the transmission probability
        if(pmax==0):
            pmax = max(transmission_prob)
            state_max = T[transmission_prob.index(pmax)]          
        else:
            state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [112]:
# accuracy using the modification 1 done to viterbi algo
start = time.time()
tagged_seq_1 = Viterbi_Mod1(test_tagged_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

# accuracy
check = [i for i, j in zip(tagged_seq_1, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq_1)
print('Modified Viterbi_Mod1 Accuracy: ',accuracy*100)

Time taken in seconds:  15.109654664993286
Modified Viterbi_Mod1 Accuracy:  93.81443298969072


Modification -2
  * <b>using a rule based tagger to backoff in case a unknown word is identified</b>

In [113]:
patterns = [
    (r'.*ed$', 'VERB'),               # past tense
    (r'.*ty$', 'ADJ'),                # Adjective
    (r'.*ing$', 'VERB'),              # gerund
    (r'.*less$', 'ADV'),              # Adverb
    (r'.*es$', 'VERB'),               # verb    
    (r'.*\'s$', 'NOUN'),              # possessive nouns
    (r'.*s$', 'NOUN'),                # plural nouns
    (r'^-?[0-9]+(.[0-9]+)?$', 'NUM'), # cardinal numbers
    (r'.*', 'NOUN'),                  # nouns
    (r'.*-130', 'X'),                 # nouns
]

# rule based tagger
rule_based_tagger = nltk.RegexpTagger(patterns)

In [114]:
def Viterbi_Mod2(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        state_max = rule_based_tagger.tag([word])[0][1]       
       
        if(pmax==0):
            state_max = rule_based_tagger.tag([word])[0][1] # assign based on rule based tagger
        else:
            if state_max != 'X':
                # getting state for which probability is maximum
                state_max = T[p.index(pmax)]                
            
        
        state.append(state_max)
    return list(zip(words, state))

In [115]:
start = time.time()
tagged_seq_2 = Viterbi_Mod2(test_tagged_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)
check = [i for i, j in zip(tagged_seq_2, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq_2)
print('Modified Viterbi_2 Accuracy: ',accuracy*100)

Time taken in seconds:  15.819526433944702
Modified Viterbi_2 Accuracy:  93.81443298969072


In [116]:
# incorrectly tagged words 
[j for i, j in enumerate(zip(tagged_seq_2, test_run_base)) if j[0] != j[1]]

[(('develops', 'NOUN'), ('develops', 'VERB')),
 (('markets', 'NOUN'), ('markets', 'VERB')),
 (('low-cost', 'NOUN'), ('low-cost', 'ADJ')),
 (('pre-1917', 'NOUN'), ('pre-1917', 'ADJ')),
 (('test', 'VERB'), ('test', 'NOUN')),
 (('out', 'PRT'), ('out', 'ADV'))]

#### Evaluating tagging accuracy

In [117]:
f = open('Test_sentences.txt')
text = f.read()
sample_test_sent = text.splitlines()
f.close()
sample_test_words = [word for sent in sample_test_sent for word in sent.split()]

In [118]:
start = time.time()
sample_tagged_seq = Viterbi(sample_test_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)
sample_tagged_seq

Time taken in seconds:  29.156334161758423


[('Android', 'NOUN'),
 ('is', 'VERB'),
 ('a', 'DET'),
 ('mobile', 'ADJ'),
 ('operating', 'NOUN'),
 ('system', 'NOUN'),
 ('developed', 'VERB'),
 ('by', 'ADP'),
 ('Google.', 'NOUN'),
 ('Android', 'NOUN'),
 ('has', 'VERB'),
 ('been', 'VERB'),
 ('the', 'DET'),
 ('best-selling', 'ADJ'),
 ('OS', 'NOUN'),
 ('worldwide', 'NOUN'),
 ('on', 'ADP'),
 ('smartphones', 'NOUN'),
 ('since', 'ADP'),
 ('2011', 'NOUN'),
 ('and', 'CONJ'),
 ('on', 'ADP'),
 ('tablets', 'NOUN'),
 ('since', 'ADP'),
 ('2013.', 'NOUN'),
 ('Google', 'NOUN'),
 ('and', 'CONJ'),
 ('Twitter', 'NOUN'),
 ('made', 'VERB'),
 ('a', 'DET'),
 ('deal', 'NOUN'),
 ('in', 'ADP'),
 ('2015', 'NOUN'),
 ('that', 'ADP'),
 ('gave', 'VERB'),
 ('Google', 'NOUN'),
 ('access', 'NOUN'),
 ('to', 'PRT'),
 ("Twitter's", 'NOUN'),
 ('firehose.', 'NOUN'),
 ('Twitter', 'NOUN'),
 ('is', 'VERB'),
 ('an', 'DET'),
 ('online', 'NOUN'),
 ('news', 'NOUN'),
 ('and', 'CONJ'),
 ('social', 'ADJ'),
 ('networking', 'NOUN'),
 ('service', 'NOUN'),
 ('on', 'ADP'),
 ('which', 'D

In [119]:
start = time.time()
sample_tagged_seq = Viterbi_Mod2(sample_test_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)
sample_tagged_seq

Time taken in seconds:  27.664135694503784


[('Android', 'NOUN'),
 ('is', 'VERB'),
 ('a', 'DET'),
 ('mobile', 'ADJ'),
 ('operating', 'NOUN'),
 ('system', 'NOUN'),
 ('developed', 'VERB'),
 ('by', 'ADP'),
 ('Google.', 'NOUN'),
 ('Android', 'NOUN'),
 ('has', 'VERB'),
 ('been', 'VERB'),
 ('the', 'DET'),
 ('best-selling', 'ADJ'),
 ('OS', 'NOUN'),
 ('worldwide', 'NOUN'),
 ('on', 'ADP'),
 ('smartphones', 'VERB'),
 ('since', 'ADP'),
 ('2011', 'NUM'),
 ('and', 'CONJ'),
 ('on', 'ADP'),
 ('tablets', 'NOUN'),
 ('since', 'ADP'),
 ('2013.', 'NOUN'),
 ('Google', 'NOUN'),
 ('and', 'CONJ'),
 ('Twitter', 'NOUN'),
 ('made', 'VERB'),
 ('a', 'DET'),
 ('deal', 'NOUN'),
 ('in', 'ADP'),
 ('2015', 'NUM'),
 ('that', 'ADP'),
 ('gave', 'VERB'),
 ('Google', 'NOUN'),
 ('access', 'NOUN'),
 ('to', 'PRT'),
 ("Twitter's", 'NOUN'),
 ('firehose.', 'NOUN'),
 ('Twitter', 'NOUN'),
 ('is', 'VERB'),
 ('an', 'DET'),
 ('online', 'NOUN'),
 ('news', 'NOUN'),
 ('and', 'CONJ'),
 ('social', 'ADJ'),
 ('networking', 'NOUN'),
 ('service', 'NOUN'),
 ('on', 'ADP'),
 ('which', 'DET

### Compare the tagging accuracies of the modifications with the vanilla Viterbi algorithm

The accuracy of vanilla Viterbi Algorithm: 92 %

The accuracy of modified Viterbi with rule base backoffad Algorithm: 94%

### List down cases which were incorrectly tagged by original POS tagger and got corrected by your modifications

The following cases were incorrectly tagged which got corrected by modified Viterbi Algorithm:
    <table>
        <tr>
            <th>Word</th>
            <th>Wrong POS tag</th>
            <th>Corrected POS tag</th>
        </tr>
        <tr>
            <td>Nonetheless</td>
            <td>Noun</td>
            <td>Adv</td>
        </tr>
        <tr>
            <td>plenty</td>
            <td>Noun</td>
            <td>ADJ</td>
        </tr>
        <tr>
            <td>lofty</td>
            <td>Noun</td>
            <td>ADJ</td>
        </tr>
    </table>